In [19]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_rows', None)
from sklearn.preprocessing import normalize

def loadFiles(path):
    furhat_files = []
    
    for filename in os.listdir(path):
        if filename.endswith('.csv'):
            src = filename
            furhat_files.append(src)
            
    return furhat_files

def returnFileGroup(lookup,furhat_pos_name):
    row = lookup[lookup['furhat_pose']==furhat_pos_name]
    fur = pd.read_csv('video_log_labels/furhat_csvs/'+row['furhat_pose'].tolist()[0])
    part = pd.read_csv('video_log_labels/participant_video_csvs/'+row['participant_pose'].tolist()[0])
    spee = pd.read_csv('video_log_labels/speech_csvs/'+row['speech'].tolist()[0])
    fs = pd.read_csv('video_log_labels/furhat_speech_csvs/'+row['furhat_speech'].tolist()[0])
    return fur, fs, part, spee

def matchTime(furhatpose, furhatspeech, particip, transc):
    try:
        time_video = float(transc[transc['DisplayText'].str.contains('Vad heter du och')]['Offset'].tolist()[0])
        time_log = float(furhatspeech[furhatspeech['value'].str.contains('text: Vad heter du')]['timestamp'].tolist()[0])
    except:
        try:
            time_video = float(transc[transc['DisplayText'].str.contains('Trevligt att')]['Offset'].tolist()[0])
            time_log = float(furhatspeech[furhatspeech['value'].str.contains('text: Trevligt att')]['timestamp'].tolist()[0])
        except:
            try:
                time_video = float(transc[transc['DisplayText'].str.contains('vad heter du?')]['Offset'].tolist()[0])
                time_log = float(furhatspeech[furhatspeech['value'].str.contains('text: Vad heter du')]['timestamp'].tolist()[0])
            except:
                time_video = float(transc[transc['DisplayText'].str.contains('Vad gör du och')]['Offset'].tolist()[0])
                time_log = float(furhatspeech[furhatspeech['value'].str.contains('text: Vad heter du')]['timestamp'].tolist()[0])
    print('video matched at '+str(time_video)+' to log at '+str(time_log))
    #hard coded with 5455
    #time_log = -1.013525
    #video_log = 28.147
    furhatspeech['timestamp'] = furhatspeech['timestamp'].astype(float)-time_log
    furhatpose['timestamp'] = furhatpose['timestamp']-time_log
    particip['time_elapsed (s)'] = particip['time_elapsed (s)']-time_video
    #particip.columns = 
    transc['Offset'] = transc['Offset'].astype(float)-time_video
    return furhatpose, furhatspeech, particip, transc

In [21]:
file_idx = 13 #choose index between 1 and 26 (not 7 because there is no 38 video)
speech = loadFiles('video_log_labels/speech_csvs/')
furhat = loadFiles('video_log_labels/furhat_csvs/')
furhat_speech = loadFiles('video_log_labels/furhat_speech_csvs/')
lookup = pd.DataFrame(pd.read_excel('video_log_labels/log_lookup.xlsx'))
print('analyzing ',furhat[file_idx])
furhat_pose, furhat_speech, participant, transcribe = returnFileGroup(lookup,furhat[file_idx])
#print(transcribe[['Offset','DisplayText']])
part_raw = participant.copy()
#print(furhat_speech)
fp_adj, fs_adj, p_adj, t_adj = matchTime(pd.DataFrame(furhat_pose), pd.DataFrame(furhat_speech).astype(str), pd.DataFrame(participant), pd.DataFrame(transcribe).astype(str))

analyzing  Sep-23-2020_15-57-49_control_054_Furhat.csv
video matched at 32.26 to log at 3.0191881656646733
      Unnamed: 0  time_elapsed (s)  left angle vertical  \
0              0          0.100000                  -16   
1              1          0.200000                  -18   
2              2          0.333333                  -22   
3              3          0.433333                  -20   
4              4          0.566667                  -16   
5              5          0.700000                  -17   
6              6          0.800000                   88   
7              7          0.933333                  -29   
8              8          1.066667                   79   
9              9          1.166667                   72   
10            10          1.300000                   74   
11            11          1.433333                  -83   
12            12          1.566667                   87   
13            13          1.666667                   47   
14      

In [43]:
from tqdm import tqdm
left_gaze = []
right_gaze = []
print(part_raw)
for i,j in tqdm(part_raw.iterrows()):
    left_val = -1
    right_val = -1
    for k,l in data.iterrows():
        if (j['time_elapsed (s)'] >= l['begin_time']) & (j['time_elapsed (s)'] <= l['end_time']):
            if l['gaze_subject'] == 'GazeOfLeft':
                if l['gaze_object'] == 'Robot':
                    left_val = 0
                elif l['gaze_object'] == 'Tablet':
                    left_val = 1
                elif l['gaze_object'] == 'Other':
                    left_val = 2
            elif l['gaze_subject'] == 'GazeOfRight':
                if l['gaze_object'] == 'Robot':
                    right_val = 0
                elif l['gaze_object'] == 'Tablet':
                    right_val = 1
                elif l['gaze_object'] == 'Other':
                    right_val = 2
    left_gaze.append(left_val)
    right_gaze.append(right_val)

21it [00:00, 208.30it/s]

      Unnamed: 0  time_elapsed (s)  left angle vertical  \
0              0          0.100000                  -16   
1              1          0.200000                  -18   
2              2          0.333333                  -22   
3              3          0.433333                  -20   
4              4          0.566667                  -16   
5              5          0.700000                  -17   
6              6          0.800000                   88   
7              7          0.933333                  -29   
8              8          1.066667                   79   
9              9          1.166667                   72   
10            10          1.300000                   74   
11            11          1.433333                  -83   
12            12          1.566667                   87   
13            13          1.666667                   47   
14            14          1.800000                   37   
15            15          1.933333                    6 

8925it [00:31, 287.29it/s]


In [53]:
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.semi_supervised import LabelPropagation

# print(left_gaze)
left_data_array = np.array(part_raw[['right angle vertical','right angle horizontal']]).reshape(len(left_gaze),2)
left_model = LabelPropagation()
left_model.fit(left_data_array, left_gaze)
pred = np.array(left_model.predict(left_data_array))

# #SEPARATE PREDICTED SAMPLES
# labels = left_gaze
# #pred = pred[left_gaze == -1]

# #PRINT CONFUSION MATRIX
# #print(confusion_matrix(labels, pred))
for i in range(len(pred)):
    print(pred[i])

C:\Users\Catherine\Anaconda3\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:281: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Catherine\Anaconda3\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:292: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
data = pd.read_csv('video_log_labels/92_annotations.txt', sep='\t', header = None)
data.columns = ['gaze_subject', 'ignore','ignore2', 'begin_time','ignore4','end_time', 'ignore6', 'duration','gaze_object']
data = data.drop(['ignore','ignore2', 'ignore4','ignore6'],1)
print(data)